In [ ]:
#default_exp s3

# S3Cache

## Deals with everything that reads and write to the s3 cache for the database

In [ ]:
#hide
import pickle, os

os.environ['DATABASE_TABLE_NAME'] = 'product-table-dev-manual'
os.environ['REGION'] = 'ap-southeast-1'
os.environ['INVENTORY_BUCKET_NAME'] = 'product-bucket-dev-manual'
os.environ['INPUT_BUCKET_NAME'] = 'input-product-bucket-dev-manual'
os.environ['DAX_ENDPOINT'] = 'longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111'
os.environ['LINEKEY'] = '2uAfV4AoYglUGmKTAk2xNOm0aV2Ufgh1BQPvQl9vJd4'
REGION = 'ap-southeast-1'

In [ ]:
#export
from s3bz.s3bz import S3
from nicHelper.wrappers import add_method, add_class_method, add_static_method
from nicHelper.dictUtil import stripDict, printDict, hashDict, saveStringToFile, loadStringFromFile, saveDictToFile, loadDictFromFile
from nicHelper.exception import errorString
import nicHelper.pdUtils as pdUtils
from dict_hash import dict_hash, sha256
from base64 import b64encode, b64decode
from pandas.util import hash_pandas_object
from hashlib import sha1
import pandas as pd
import os, logging

In [ ]:
from villaProductDatabase.database import ProductDatabase
import logging
logging.basicConfig(level=logging.INFO)

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111
longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
import os
DBHASHLOCATION = '/mnt/efs/database.hash'
DBCACHELOCATION = '/mnt/efs/database.cache'
DATABASE_TABLE_NAME = os.environ.get('DATABASE_TABLE_NAME')
INVENTORY_BUCKET_NAME = os.environ.get('INVENTORY_BUCKET_NAME')
INPUT_BUCKET_NAME = os.environ.get('INPUT_BUCKET_NAME')
REGION = os.environ.get('REGION') or 'ap-southeast-1'
ACCESS_KEY_ID = os.environ.get('USER') or None
SECRET_ACCESS_KEY = os.environ.get('PW') or None
LINEKEY= os.environ.get('LINEKEY')
ALLDATAKEY = 'allData'
  
try:
  DAX_ENDPOINT = os.environ['DAX_ENDPOINT']
  print(DAX_ENDPOINT)
except KeyError as e:
  print(f'dax endpoint missing {e}')
  

longtermcluster.vuu7lr.clustercfg.dax.apse1.cache.amazonaws.com:8111


In [ ]:
#export
class S3Cache:
  pass

In [ ]:
class Tester( S3Cache, ProductDatabase):
  class Meta:
    table_name = os.environ['DATABASE_TABLE_NAME']
    region = os.environ['REGION']
    billing_mode='PAY_PER_REQUEST'
    dax_read_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
    dax_write_endpoints = [DAX_ENDPOINT] if DAX_ENDPOINT else None
  pass
  

## save and load local hash

In [ ]:
#export
@add_class_method(S3Cache)
def loadFromCache(cls, key=ALLDATAKEY, localCache=DBCACHELOCATION, 
                  localHash=DBHASHLOCATION, bucket=INVENTORY_BUCKET_NAME):
  ## check for local object and its hash
  if os.path.exists(localCache) and os.path.exists(localHash):
    localHash = pdUtils.loadLocalHash(path=localHash)
    remoteHash = pdUtils.loadRemoteHash(key=key, bucket=bucket)
    if localHash == remoteHash:
      print('data is still in sync, using local cache')
      db = pdUtils.loadLocalCache(path=localCache)
      return db
    else: 
      print('remote hash is not the same, load remote cache')
  ### load from remote cache
  db = pdUtils.loadRemoteCache(key=key, bucket=bucket)
  ### save to local cache
  pdUtils.saveLocalCache(db, path=localCache)
  pdUtils.saveLocalHash(db, path = localHash)
  return db

In [ ]:
%%time
db = Tester.loadFromCache()
assert db.shape > (10,10)

INFO:root:using accelerate endpoint


loading hashkey allData-hash


INFO:root:object exists, loading
INFO:root:using accelerate endpoint


loaded hash is e706d6d6b6a9a7e831045ed1e859e7fb24bd5e4b
data is still in sync, using local cache
CPU times: user 315 ms, sys: 36.8 ms, total: 351 ms
Wall time: 746 ms


## Save and load remote hash

In [ ]:
#export

@add_class_method(S3Cache)
def saveRemoteCache(cls ,db:pd.DataFrame, key= ALLDATAKEY,
                   bucket = INVENTORY_BUCKET_NAME, localCachePath=DBCACHELOCATION,
                   localHashPath=DBHASHLOCATION):
  pdUtils.saveRemoteCache(data=db, key= key, 
                          bucket=bucket, localCachePath=localCachePath, 
                          localHashPath=localHashPath)


In [ ]:
%%time
Tester.saveRemoteCache(db)
assert pdUtils.getDfHash(db) == pdUtils.getDfHash(Tester.loadFromCache())

INFO:root:using accelerate endpoint
INFO:root:data was saved to s3
INFO:root:using accelerate endpoint


hashKey is allData-hash
saving hash to s3
saved hash e706d6d6b6a9a7e831045ed1e859e7fb24bd5e4b


INFO:root:using accelerate endpoint


loading hashkey allData-hash


INFO:root:object exists, loading
INFO:root:using accelerate endpoint


loaded hash is e706d6d6b6a9a7e831045ed1e859e7fb24bd5e4b
data is still in sync, using local cache
CPU times: user 2.1 s, sys: 439 ms, total: 2.54 s
Wall time: 3.99 s
